# # 🚀 Sistema de copy trading

In [ ]:
import asyncio
import os

from copy_trading import (
    CopyTrading,
    CopyTradingConfig
)

# Archivo de configuración
file_path = os.getenv("CONFIG_FILE_PATH", "copy_trading/configs_in_json/config.json")

In [ ]:
async def main():
    """Función principal para ejecutar el sistema con notificaciones."""
    # 1. Cargar la configuración del bot

    config = CopyTradingConfig.load_from_file(file_path)

    # 2. Crear e iniciar el sistema
    system = CopyTrading(config=config)

    try:
        async with system:
            print(f"\n✅ Sistema iniciado con notificaciones")
            print(f"🔄 Modo: {'DRY RUN' if config.dry_run else 'LIVE'}")
            print(f"👥 Siguiendo {len(config.traders)} traders")
            print("\nPresiona Ctrl+C para detener.")

            # Enviar notificación de prueba
            if system.notification_manager:
                status_msg = (
                    f"Sistema iniciado correctamente\n"
                    f"- Modo: {'🔄 DRY RUN' if config.dry_run else '🚀 LIVE'}\n"
                    f"- Traders: 👥 {len(config.traders)}"
                )
                await system.notification_manager.notify_system(status_msg, "success")

            # Bucle principal - mostrar métricas cada 5 minutos
            while system.is_running:
                await asyncio.sleep(300)  # 5 minutos

                try:
                    metrics = await system.get_metrics()

                    print("\n--- Métricas del Sistema ---")
                    print(f"  - Tiempo activo: {metrics['system_metrics']['uptime_seconds']:.0f}s")
                    print(f"  - Trades recibidos: {metrics['callback_stats']['trades_received']}")
                    print(f"  - Trades validados: {metrics['callback_stats']['trades_validated']}")
                    print(f"  - Trades en cola: {metrics['queue_stats']['pending_count']}")
                    print(f"  - Trades ejecutados: {metrics['system_metrics']['trades_executed']}")
                    print(f"  - Volumen total: {metrics['system_metrics']['total_volume_sol']:.6f} SOL")
                    print(f"  - Balance actual: {metrics['wallet_balance']:.6f} SOL")
                    print("--------------------------\n")

                    # Enviar resumen por Telegram cada hora
                    if system.notification_manager and metrics['system_metrics']['uptime_seconds'] % 3600 < 300:
                        metrics_msg = (
                            f"Resumen horario del sistema\n"
                            f"- Tiempo activo: ⏱️ {metrics['system_metrics']['uptime_seconds']/3600:.1f}h\n"
                            f"- Trades recibidos: 📥 {metrics['callback_stats']['trades_received']}\n"
                            f"- Trades ejecutados: ✅ {metrics['system_metrics']['trades_executed']}\n"
                            f"- Volumen: 💰 {metrics['system_metrics']['total_volume_sol']:.6f} SOL\n"
                            f"- Balance: 💎 {metrics['wallet_balance']:.6f} SOL"
                        )
                        await system.notification_manager.notify_system(metrics_msg, "info")

                except Exception as e:
                    print(f"Error obteniendo métricas: {e}")

    except KeyboardInterrupt:
        print("\nInterrupción del usuario detectada.")
    except Exception as e:
        print(f"Error en el sistema: {e}")

        # Notificar error crítico
        if system.notification_manager:
            error_msg = (
                f"Error crítico en el sistema:\n"
                f"- Tipo: ⚠️ {type(e).__name__}\n"
                f"- Mensaje: 🔥 {str(e)}"
            )
            await system.notification_manager.notify_system(error_msg, "error")

    print("🏁 Sistema detenido.")

In [ ]:
try:
    await main()
except asyncio.CancelledError:
    print("Programa cancelado por el usuario")